<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/POC_LLM_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/collections/open-llm-leaderboard/the-big-benchmarks-collection-64faca6335a7fc7d4ffe974a


https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration

In [1]:
!nvidia-smi

Fri Apr  5 08:05:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   59C    P8              14W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

https://www.engineering.com/story/design-and-engineering-performance-and-flexibility-the-nvidia-l4-tensor-core-gpu

In [ ]:
!pip install colab-env --quiet
!pip install tiktoken -q
!pip install accelerate -q

In [3]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Mounted at /content/gdrive
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "davidkim205/Rhea-72b-v0.5" # HF AVERAGE = 81.22 #1 APRIL 5TH, 2024
#Metric	Value
#Avg.	81.22
#AI2 Reasoning Challenge (25-Shot)	79.78
#HellaSwag (10-Shot)	91.15
#MMLU (5-Shot)	77.95
#TruthfulQA (0-shot)	74.50
#Winogrande (5-shot)	87.85
#GSM8k (5-shot)	76.12


tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True, do_sample=True)

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [8]:
#prompt="What was the first album Beyoncé released as a solo artist?"
prompt="What is the capital of canada?"
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.1,top_k=50, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)

In [9]:
print('Question: %s'%prompt)
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Question: What is the capital of canada?
Generated Answer:
The capital of Canada is Ottawa.
